# code feedback

In [1]:
# 中文sft有350w，预训练英文比例为20%，需要70w英文。(4:1：应该是350/4)英文sft数据总共有340w，采样70/340
SAMPLE_RATE = 70/340.0

In [2]:
import json
from tqdm import tqdm
def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 去掉行末的换行符，并解析为 JSON 对象
            json_obj = json.loads(line.strip())
            yield json_obj

In [6]:
import pandas as pd
import os
from tqdm import tqdm
import json
import numpy as np

def convert_parquet_to_jsonl(parquet_file_path):
    """
    将.parquet格式的文件转换为.jsonl格式
    
    参数:
        parquet_file_path: parquet文件的路径
    
    返回:
        jsonl_file_path: 保存的jsonl文件路径
    """
    print(f"正在读取parquet文件: {parquet_file_path}")
    # 读取parquet文件
    df = pd.read_parquet(parquet_file_path)
    
    # 构建输出文件路径（与输入文件相同路径，但扩展名改为.jsonl）
    jsonl_file_path = os.path.splitext(parquet_file_path)[0] + '.jsonl'
    
    print(f"正在将数据转换并保存为jsonl: {jsonl_file_path}")
    # 将DataFrame转换为jsonl格式并保存
    with open(jsonl_file_path, 'w', encoding='utf-8') as f:
        for _, row in tqdm(df.iterrows(), total=len(df), desc="转换进度"):
            #! 将每一行转换为字典
            row_dict = row.to_dict()

            #! 将numpy数组转换为列表
            for key, value in row_dict.items():
                if isinstance(value, np.ndarray):
                    row_dict[key] = value.tolist()

            # 将每一行转换为JSON字符串并写入文件
            json.dump(row_dict, f, ensure_ascii=False)
            f.write('\n')
    
    print(f"转换完成! 文件已保存至: {jsonl_file_path}")
    return jsonl_file_path

# 示例使用
parquet_file_path = "/DATA/disk2/yuhang/.cache/modelscope/datasets/HuggingFaceH4/Code-Feedback/data/train_sft-00000-of-00001.parquet"
jsonl_file_path = convert_parquet_to_jsonl(parquet_file_path)


正在读取parquet文件: /DATA/disk2/yuhang/.cache/modelscope/datasets/HuggingFaceH4/Code-Feedback/data/train_sft-00000-of-00001.parquet
正在将数据转换并保存为jsonl: /DATA/disk2/yuhang/.cache/modelscope/datasets/HuggingFaceH4/Code-Feedback/data/train_sft-00000-of-00001.jsonl


转换进度: 100%|██████████| 65383/65383 [00:06<00:00, 10158.40it/s]


转换完成! 文件已保存至: /DATA/disk2/yuhang/.cache/modelscope/datasets/HuggingFaceH4/Code-Feedback/data/train_sft-00000-of-00001.jsonl


In [7]:
data_dir = "/DATA/disk2/yuhang/.cache/modelscope/datasets/HuggingFaceH4/Code-Feedback/data/train_sft-00000-of-00001.jsonl"
code_feedback_save_data = []
for json_obj in read_jsonl(data_dir):
    conversations_list = []
    message_list = json_obj["messages"]
    for  message in message_list:
        result = {}
        if message["role"]=='user':
            result['from'] = 'human'
        elif message["role"]=='assistant':
            result["from"] = 'gpt'
        else:
            raise NameError(message['role'])
        result['value'] = message['content']
        conversations_list.append(result)
    code_feedback_save_data.append({
        "conversations": conversations_list
    })
print(len(code_feedback_save_data))

65383


In [15]:
print(code_feedback_save_data[2])
file_path = '/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input/code_feedback.jsonl'

directory = os.path.dirname(file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"目录{directory} 不存在，已创建")
else:
    print(f"目录{directory} 已存在")
# 保存为JSON Lines文件
with open(file_path, 'w', encoding='utf-8') as f:
    for item in code_feedback_save_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

{'conversations': [{'from': 'human', 'value': 'I have this problem : Strange Printer II\nDescription are as below: There is a strange printer with the following two special requirements:\nOn each turn, the printer will print a solid rectangular pattern of a single color on the grid. This will cover up the existing colors in the rectangle.\n\nOnce the printer has used a color for the above operation, the same color cannot be used again.\n\nYou are given a `m x n` matrix `targetGrid`, where `targetGrid[row][col]` is the color in the position `(row, col)` of the grid.\n\nReturn `true` if it is possible to print the matrix `targetGrid`, otherwise, return `false`.\n\nAdditionally, you are given a `m x n` matrix `initialGrid`, where `initialGrid[row][col]` is the initial color in the position `(row, col)` of the grid before any printing operation. You need to consider the `initialGrid` while determining if it is possible to print the `targetGrid`.\n\nExample 1:\nInput: initialGrid = [[0,0,0,

In [14]:
print(code_feedback_save_data[2])
import random
random.seed(666)
sample_code_feedback_save_data = random.sample(code_feedback_save_data, int(len(code_feedback_save_data)*SAMPLE_RATE))
print(f"sampled: {len(sample_code_feedback_save_data)} all:{len(code_feedback_save_data)}")
# 保存为JSON Lines文件
with open('/data/data/fintuning/llamafactory_input/code_feedback_random_20%.jsonl', 'w', encoding='utf-8') as f:
    for item in sample_code_feedback_save_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

{'conversations': [{'from': 'human', 'value': 'I have this problem : Strange Printer II\nDescription are as below: There is a strange printer with the following two special requirements:\nOn each turn, the printer will print a solid rectangular pattern of a single color on the grid. This will cover up the existing colors in the rectangle.\n\nOnce the printer has used a color for the above operation, the same color cannot be used again.\n\nYou are given a `m x n` matrix `targetGrid`, where `targetGrid[row][col]` is the color in the position `(row, col)` of the grid.\n\nReturn `true` if it is possible to print the matrix `targetGrid`, otherwise, return `false`.\n\nAdditionally, you are given a `m x n` matrix `initialGrid`, where `initialGrid[row][col]` is the initial color in the position `(row, col)` of the grid before any printing operation. You need to consider the `initialGrid` while determining if it is possible to print the `targetGrid`.\n\nExample 1:\nInput: initialGrid = [[0,0,0,

FileNotFoundError: [Errno 2] No such file or directory: '/data/data/fintuning/llamafactory_input/code_feedback_random_20%.jsonl'

# open hermes

In [12]:
import json
# 读取JSON文件
data_dir = "/DATA/disk2/yuhang/.cache/modelscope/datasets/swift/OpenHermes-2___5/openhermes2_5.json"
with open(data_dir, 'r', encoding='utf-8') as f:
    data = json.load(f)
# 打印读取的数据
print(len(data))

1001551


In [13]:
print(data[0])
with open('/DATA/disk2/yuhang/.cache/modelscope/datasets/swift/OpenHermes-2___5/openhermes2_5.json', 'w', encoding='utf-8') as f:
    for item in data:
        save_item = {"conversations":item['conversations']}
        json.dump(save_item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象
    

{'conversations': [{'from': 'human', 'value': 'Every day, a tree drops 7 leaves. How many leaves would it drop in a month of February in a non-leap year? Include your logic.'}, {'from': 'gpt', 'value': "Here's the logic behind this:\n\n1. We know that February has 28 days in a non-leap year.\n2. If the tree drops 7 leaves every day, then over the course of February, it would drop:\n   Leaves dropped in February = Leaves per day * Days in February\n   = 7 leaves * 28 days\n   = 196 leaves\n\nSo, the tree would drop 196 leaves in February in a non-leap year."}], 'source': 'airoboros2.2', 'category': 'orca', 'skip_prompt_formatting': False}


In [16]:
print(data[0])
import random
random.seed(666)
random_data = random.sample(data, int(len(data)*SAMPLE_RATE))
print(f"sampled: {len(random_data)} all:{len(data)}")

file_path = '/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input/openhermes25.jsonl'

directory = os.path.dirname(file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"目录{directory} 不存在，已创建")
else:
    print(f"目录{directory} 已存在")

with open(file_path, 'w', encoding='utf-8') as f:
    for item in random_data:
        save_item = {"conversations":item['conversations']}
        json.dump(save_item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

{'conversations': [{'from': 'human', 'value': 'Every day, a tree drops 7 leaves. How many leaves would it drop in a month of February in a non-leap year? Include your logic.'}, {'from': 'gpt', 'value': "Here's the logic behind this:\n\n1. We know that February has 28 days in a non-leap year.\n2. If the tree drops 7 leaves every day, then over the course of February, it would drop:\n   Leaves dropped in February = Leaves per day * Days in February\n   = 7 leaves * 28 days\n   = 196 leaves\n\nSo, the tree would drop 196 leaves in February in a non-leap year."}], 'source': 'airoboros2.2', 'category': 'orca', 'skip_prompt_formatting': False}
sampled: 206201 all:1001551
目录/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input 已存在


# WebInstructSub

In [2]:
import pyarrow.parquet as pq
from glob import glob
all_file_dirs = glob("/data/data/fintuning/WebInstructSub/data/*parquet")
webinsturt_all_data = []
for file_dir in all_file_dirs:
    table = pq.read_table(file_dir)
    print(len(table))
    df = table.to_pandas()
    question_list = df["question"]
    answer_list = df["answer"]
    L = len(question_list)
    for i in range(L):
        webinsturt_all_data.append(
            {
                "instruction":question_list.iloc[i],
                "input":"",
                "output": answer_list.iloc[i]
            }
        )

179632
179632
179633
179632
179633
179632
179633
179632
179632
179632
179632
179633
179632


In [7]:
print(len(webinsturt_all_data), webinsturt_all_data[1])
with open('/data/data/fintuning/llamafactory_input/webinstruct_english.jsonl', 'w', encoding='utf-8') as f:
    for item in webinsturt_all_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象
    

2335220 {'instruction': 'I recently became fascinated by infinite nested radicals, first drawn attention to me from a question in my textbook about the value of $\\sqrt{1+\\sqrt{{1}+\\sqrt{{1}+\\sqrt{{1}...}}}}$ which turned out to be $\\phi$ when I worked it out, a rather beautiful result. I then tried to find a formula to evaluate the general case $$\\sqrt{x+\\sqrt{{x}+\\sqrt{{x}+\\sqrt{{x}...}}}}$$ which I succeeded in; it can be evaluated as $$\\frac{1+\\sqrt{1+4x}}{2}$$ Multiplying the nested radical which was equal to $\\phi$ by $x$ produces the following nested radical: $$\\sqrt{{x^2}+\\sqrt{{x^4}+\\sqrt{{x^8}+\\sqrt{{x^{16}}...}}}}$$ so this is equal to $x\\left(\\frac{1+\\sqrt5}{2}\\right)$. However, I have tried and failed to find the value of the following infinite square root: $$\\sqrt{x+\\sqrt{{x^2}+\\sqrt{{x^3}+\\sqrt{{x^4}...}}}}$$', 'input': '', 'output': 'The function of the OP is denoted as $f_1(x)$. For $0\\lt x\\ll 1$, we have $f_1(x)\\approx 1$. For large positive 

In [ ]:
# random 20%
import random
random.seed(666)
random_data = random.sample(webinsturt_all_data, int(len(webinsturt_all_data)*SAMPLE_RATE))
print(f"sampled: {len(random_data)} all:{len(data)}")
print(len(webinsturt_all_data), webinsturt_all_data[1])
with open('/data/data/fintuning/llamafactory_input/webinstruct_english.jsonl', 'w', encoding='utf-8') as f:
    for item in webinsturt_all_data:
        json.dump(item, f, ensure_ascii=False)  # ensure_ascii=False 确保中文不乱码
        f.write('\n')  # 每行一个JSON对象

# openbookqa & ai2_arc & logiqa

In [1]:
import os
import json
import pyarrow.parquet as pq
def format_choices(question, choice_list, answer, answer_idx):
    example = (
        "The following is a multiple-choice question. Please choose the most suitable one among A, B, C and D as the answer to this question.\n\n"
        + question
        + "\n"
    )
    for idx, choice in enumerate(["A", "B", "C", "D"]):
        example += f'{choice}. {choice_list[idx]}\n'
    ouput = f'{answer}. {choice_list[answer_idx]}'
    return example, ouput

In [11]:
# openbookqa
# https://huggingface.co/datasets/allenai/openbookqa

openbookqa_all_data = []
openbookqa_root_dir = "/data/data/fintuning/openbookqa"
file_list = ["main/train-00000-of-00001.parquet","main/test-00000-of-00001.parquet","main/validation-00000-of-00001.parquet",
    "additional/train-00000-of-00001.parquet","additional/test-00000-of-00001.parquet","additional/validation-00000-of-00001.parquet"]
for half_file_dir in file_list:
    file_dir = os.path.join(openbookqa_root_dir, half_file_dir)
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    for i in range(len(df)):
        question = df.iloc[i]["question_stem"]
        answer = df.iloc[i]["answerKey"]
        choice_list = df.iloc[i]["choices"]["text"]
        answer_idx = ord(answer) - ord('A')
        example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
        openbookqa_all_data.append(
            {
                "instruction":example,
                "input":"",
                "output": final_answer
            }
        )

In [12]:
print(len(openbookqa_all_data), openbookqa_all_data[3])

11914 {'instruction': 'The following is a multiple-choice question. Please choose the most suitable one among A, B, C and D as the answer to this question.\n\nStars are\nA. warm lights that float\nB. made out of nitrate\nC. great balls of gas burning billions of miles away\nD. lights in the sky\n', 'input': '', 'output': 'C. great balls of gas burning billions of miles away'}


In [10]:
# ai2_arc
# https://huggingface.co/datasets/allenai/ai2_arc
ai2_arc_all_data = []
ai2_arc_root_dir = "/data/data/fintuning/ai2_arc"
file_list = ["ARC-Challenge/train-00000-of-00001.parquet","ARC-Challenge/test-00000-of-00001.parquet","ARC-Challenge/validation-00000-of-00001.parquet",
    "ARC-Easy/train-00000-of-00001.parquet","ARC-Easy/test-00000-of-00001.parquet","ARC-Easy/validation-00000-of-00001.parquet"]
error_counter = 0
num_to_char = {"1":"A", "2":"B", "3":"C", "4":"D"}
for half_file_dir in file_list:
    file_dir = os.path.join(ai2_arc_root_dir, half_file_dir)
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    for i in range(len(df)):
        question = df.iloc[i]["question"]
        answer = df.iloc[i]["answerKey"]
        choice_list = df.iloc[i]["choices"]["text"]
        answer_idx = ord(answer) - ord('A')
        try:
            example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
        except:
            # answer_idx = int(answer)-1
            # # 1-4
            # answer = num_to_char[answer]
            # example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
            print(question, choice_list, answer, answer_idx)
            error_counter += 1
        ai2_arc_all_data.append(
            {
                "instruction":example,
                "input":"",
                "output": final_answer
            }
        )
print(len(ai2_arc_all_data), error_counter)

A chemical property of a mineral is evident if the mineral ['breaks easily when struck with a hammer'
 'bubbles when acid is placed on it' 'is easily scratched by a fingernail'
 'reflects light from its surface'] 2 -15
Several tomato plants are grown indoors next to a sunny window. The plants receive water and fertilizer and remain on the windowsill. What will most likely happen? ['Most of the leaves on the window side will wilt and die.'
 'The roots of the plants will grow upward from the soil.'
 'Water droplets will collect on the leaves facing away from the window.'
 'The stem will bend toward the window.'] 4 -13
Weather forecasts are more accurate today than in the past due to ['global warming' 'air-quality control' 'plate tectonics'
 'use of images from space'] 4 -13
Which sequence of energy transformations occurs after a battery-operated flashlight is turned on? ['electrical -> light -> chemical' 'electrical -> chemical -> light'
 'chemical -> light -> electrical' 'chemical -> el

In [ ]:
logiqa_all_data = []
logiqa_file_dirs = ["/data/data/fintuning/LogiQA2.0/logiqa/DATA/LOGIQA/train.txt",
                   "/data/data/fintuning/LogiQA2.0/logiqa/DATA/LOGIQA/dev.txt",
                   "/data/data/fintuning/LogiQA2.0/logiqa/DATA/LOGIQA/test.txt",
                   ]
num_to_char = {0:"A", 1:"B", 2:"C", 3:"D"}
for file_dir in logiqa_file_dirs:
    # 打开文件
    with open(file_dir, 'r', encoding='utf-8') as file:
        # 读取所有行
        lines = file.readlines()
    for line in lines:
        json_data = json.loads(line.strip())  # 使用 strip() 去除行末的换行符
        try:
            question = f"context:{json_data['context']}\n question:{json_data['query']}"
        except:
            print(json_data.keys())
            continue
        answer_idx = int(json_data["correct_option"])
        answer = num_to_char[answer_idx]
        choice_list = json_data["options"]
        try:
            example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
        except:
            # answer_idx = int(answer)-1
            # # 1-4
            # answer = num_to_char[answer]
            # example, final_answer = format_choices(question, choice_list, answer, answer_idx) 
            print(question, choice_list, answer, answer_idx)
            error_counter += 1
        ai2_arc_all_data.append(
            {
                "instruction":example,
                "input":"",
                "output": final_answer
            }
        )

dict_keys(['id', 'answer', 'text', 'question', 'options', 'type'])


KeyError: 'context'